<a href="https://colab.research.google.com/github/pandov/cups-mail/blob/master/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
import torch
from src.nn import BACTERIA, Runner, get_classification_components, get_loaders, score_clf
logdir = './logs/classification'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
class MultiRunner(Runner):
    def _handle_batch(self, batch):
        x, y = batch
        y_pred = self.model(x)
        is_train = self.is_train_loader == True

        with torch.set_grad_enabled(is_train):
            loss = self.criterion(y_pred, y)
            precision = score_clf(y_pred, y)
            self.batch_metrics.update({
                'precision/_mean': precision.mean(),
                'precision/_sum': precision.sum(),
                'loss': loss,
                'lr': self.state.scheduler.get_last_lr()[0],
            })

            if is_train:
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
            else:
                if self.loader_batch_step == self.loader_len:
                    self.scheduler.step()
        
        self.input = {'targets': y}
        self.output = {'logits': y_pred}

In [ ]:
net = 'resnet50'
components = get_classification_components(net, 'adam', 'steplr', weightable=True)
components['loaders'] = get_loaders(keys=['image', 'label'], batch_size=12, is_resized=True)
runner = MultiRunner()
runner.train(
    logdir=f'{logdir}/{net}',
    num_epochs=140,
    # verbose=True,
    **components
)

In [ ]:
logzip = logdir + '.zip'
! zip -r {logzip} {logdir}
! mv {logzip} 'My Drive/drive/'
# ! git pull origin master
# ! git add logs
# ! git commit -m 'Changed from Colab'
# ! git push -u origin master